In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [2]:
symbols = ["H", "H", "H"]
coordinates = np.array([[0.0102,0.0442,0.0], [0.9867,1.6303,0.0], 
                       [1.8720,-0.0085,0.0]])


In [3]:
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols,
                                                  coordinates,
                                                 charge=1)

In [4]:
print(qubits)

6


In [5]:
hf = qchem.hf_state(electrons = 2, orbitals = 6)

In [6]:
print(hf)

[1 1 0 0 0 0]


In [7]:
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)

In [8]:
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(hamiltonian)

In [9]:
exp_energy(hf)

tensor(-1.24655016, requires_grad=True)

In [10]:
def ansatz(params):
    qml.BasisState(hf,wires=range(num_wires))
    qml.DoubleExcitation(params[0],wires=[0,1,2,3])
    qml.DoubleExcitation(params[1],wires=[0,1,4,5])

In [11]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

In [12]:
cost_function([0.1,0.1])

tensor(-1.26796721, requires_grad=True)

In [14]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array([0.0,0.0], requires_grad=True)

# store the values of the cost function
energy = [cost_function(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 20


for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)

    energy.append(cost_function(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

Step = 0,  Energy = -1.26070025 Ha
Step = 2,  Energy = -1.27115671 Ha
Step = 4,  Energy = -1.27365804 Ha
Step = 6,  Energy = -1.27425241 Ha
Step = 8,  Energy = -1.27439362 Ha
Step = 10,  Energy = -1.27442718 Ha
Step = 12,  Energy = -1.27443517 Ha
Step = 14,  Energy = -1.27443707 Ha
Step = 16,  Energy = -1.27443752 Ha
Step = 18,  Energy = -1.27443763 Ha


In [15]:
print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Final angle parameters = {theta[0]:.8f} {theta[1]:.8f}")


Final value of the ground-state energy = -1.27443764 Ha

Final angle parameters = 0.19203468 0.19290335


In [16]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

In [17]:
ground_state(theta)

tensor([ 0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09585862+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
        -0.09586987+0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.99076743+0.j,  0.        +0.j,  0.        +0.

/ground state > = 0.9908/110000> - 0.096/001100> - 0.096/000011>